# Fuentes

### Link: https://heartbeat.fritz.ai/5-regression-loss-functions-all-machine-learners-should-know-4fb140e9d4b0
En esta fuente se puede encontrar una breve explicación del MAE y del MSE, una comparación entre ambos respecto de su comportamiento en entrenamiento frente a conjuntos de datos con y sin outliers, y luego una comparación de su comportamiento durante entrenamiento a razón de cómo son sus gradientes, lo cual provoca en el caso del MAE que la convergencia sea más lenta y sea necesario utilizar un **learning rate dinámico**. Explica que, si nos importa que la presencia de outliers tenga un impacto directo sobre el modelo, deberíamos utilizar MSE, mientras que si deseamos que no afecte demasiado podemos emplear MAE.

### Link: https://towardsdatascience.com/learning-rate-schedules-and-adaptive-learning-rate-methods-for-deep-learning-2c8f433990d1
En esta fuente se puede encontrar una explicación de los tres métodos para learning rate dinámico utilizados, el **time-based decay**, el **step decay** y el **exponential decay**, empleando para algunos de ellos la clase de Keras llamada Learning Rate Scheduler, que permite modificar a gusto del usuario el valor del learning rate a través del proceso.

### Link: https://stackoverflow.com/questions/46308374/what-is-validation-data-used-for-in-a-keras-sequential-model
Esta disución de StackOverflow es interesante sobre la separación de los datasets en entrenamiento, validación y evaluación del modelo, la use para verificar algunas cuestiones sobre cómo usaba la información de validación Keras, entre otras cosas.

### Link: https://machinelearningmastery.com/how-to-stop-training-deep-neural-networks-at-the-right-time-using-early-stopping/
Explicación sobre el uso de **early stopping**, donde básicamente buscamos parar el entrenamiento aunque no se hayan terminado de correr todos los epochs predefinidos, porque se detecta que no hay mejoría en los resultados obtenidos, para ello se emplea la métrica evaluada sobre el conjunto de validación.

### Link: https://machinelearningmastery.com/polynomial-features-transforms-for-machine-learning/
Explicación sobre el uso de **features polinomiales**, que básicamente consiste en agregar nuevas variables de entrada al modelo a partir de potencias obtenidas entre las variables de entrada originales. De esta forma, el espacio que conforman las variables es de mayor dimensión y por ello la solución es más flexible, aunque hay que tener cuidado de que no se ajuste demasiado provocando **overfitting**.

# 1. Cargando base de datos

In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
import numpy as np

In [3]:
import importlib

In [4]:
import sys

In [5]:
sys.path.insert(0, '../..')

In [6]:
# Read the database from the .csv file into a pandas dataframe
df = pd.read_csv('../../databases/insurance.csv')

# 2. Preprocesamiento de los datos

In [7]:
from sklearn import preprocessing

## 2.1. Codificación de variables no numéricas

In [8]:
# Create a label encoder for the sex variable or feature and create a new column in the dataframe 
# with the encoded version of the gender
sex_encoder = preprocessing.LabelEncoder()
sex_encoder.fit(df['sex'])
df['sex-encoded'] = sex_encoder.transform(df['sex'])

In [9]:
# Create a label encoder for the smoker variable or feature and create a new column in the dataframe
# with the encoded version of the smoker
smoker_encoder = preprocessing.LabelEncoder()
smoker_encoder.fit(df['smoker'])
df['smoker-encoded'] = smoker_encoder.transform(df['smoker'])

In [10]:
# Create a one hot encoder and fit the available types of regions in the dataset
region_encoder = preprocessing.OneHotEncoder()
region_encoder.fit(df['region'].to_numpy().reshape(-1, 1))

# Transform all entries into the one hot encoded representation
encoded_regions = region_encoder.transform(df['region'].to_numpy().reshape(-1, 1)).toarray()

# Add each new encoded variable or feature to the dataset
for i, category in enumerate(region_encoder.categories_[0]):
    df[f'{category}-encoded'] = encoded_regions.transpose()[i]

## 2.2. Filtrado de variables

In [11]:
# Filtering or removing of non desired variables
df_x = df[['age', 'bmi', 'smoker-encoded', 'children', 'sex-encoded', 'northwest-encoded', 'northeast-encoded', 'southwest-encoded', 'southeast-encoded']]
df_y = df['charges']

# 3. Separación del conjunto de entrenamiento y evaluación

In [12]:
from sklearn import model_selection

In [13]:
from sklearn import preprocessing

## 3.1. Separación de los conjuntos
Es importante notar que, se realiza la separación del conjunto de datos original en **train**, **valid** y **test**, por fuera del framework de Keras para garantizar un adecuado tratamiento de los conjuntos acorde a la metodología empleada. En otras palabras, de esta forma nos aseguramos que cualquier preprocesamiento o normalización sobre validación (valid) y evaluación (test) se realiza a partir de la información obtenida en entrenamiento.

In [14]:
# Split the dataset into train_valid and test
x_train_valid, x_test, y_train_valid, y_test = model_selection.train_test_split(df_x, df_y, test_size=0.2, random_state=30, shuffle=True)

In [15]:
# Split the dataset into train and valid
x_train, x_valid, y_train, y_valid = model_selection.train_test_split(x_train_valid, y_train_valid, test_size=0.3, random_state=40, shuffle=True)

# 4. Regresión Lineal


#### Comentarios
1. Al principio, sucedió que el MAE era muy lento para convergencia, lo cual tiene sentido por el tipo de función de costo que representa. Particularmente, comparado con MSE, es mucho más lentro. Empecé probando modificar de forma estática y a mano el **learning rate**.
2. Luego, con un learning rate cada vez mayor, pude observar que el entrenamiento era más rápido, pero sucedían dos cuestiones. En primer lugar, que se producía una especie oscilación en torno a un valor que asumo que es el mínimo al cual se acerca el entrenamiento, con lo cual sería necesario disminuir cerca de ahí el valor del learning rate. Por otro lado, este mínimo no era el mismo mínimo que obtuve con el MSE, debe ser un plateau, un mínimo local pero no el absoluto. Me propuse usar **learning rate dinámico** y **comenzar de diferentes puntos**.
3. Cuando probe utilizar MSE, si no normalizaba con z-score todas las variables, rápidamente divergía la función de costo y se rompía el entrenamiento. Por otro lado, la misma normalización afectaba mucho al entrenamiento del MAE. *¿Por qué?* Lo pude corregir un poco al aumentar el learning rate por un factor, lo cual debe tener sentido si se considera que ahora las variables estando normalizadas tienen una menor magnitud lo cual puede producir que los pasos sean menores que antes, y por eso se ralentizó.
4. Interesante, llegué a esta discusión https://datascience.stackexchange.com/questions/9020/do-i-have-to-standardize-my-new-polynomial-features a raiz de una pregunta bastante sencilla, **¿por qué no está mejorando la métrica con mayor orden de polynomial features?**. Resulta ser que normalizando las variables y luego aplicando polynomial features, obtengo nuevas variables que siguen encontrándose en el intervalo [0,1] pero que su orden de magnitud es mucho menor. *Conclusión, siempre normalizar las variables que entran al modelo, y por ende si aplicas polynomial features tenés que normalizar luego de crear las nuevas variables.*
5. Con la corrección mencionada anteriormente con respecto a la normalización, mejoró el resultado de ordenes grandes de polinomios.
6. Me llama la atención que por lo general los resultados de validación son mejores que en entrenamiento, y además, esta diferencia se achica más a medida que aumenta el orden de los polinomios. Me hace pensar que por alguna razón estoy en underfitting, o estimando incorrectamente las métricas (por ejemplo por tamaño del dataset). Este artículo menciona algo que puede ser útil https://keras.io/getting_started/faq/#why-is-my-training-loss-much-higher-than-my-testing-loss, una posibilidad sería que la validación sobre un epoch siempre tienda a ser mejor que el promedio del train en los batch, porque fue entrenándose mejor. Aunque no me convence después de muchos epochs que suceda esto. **¿Debería estar usando k-folding?**

In [16]:
from src import rl_helper
importlib.reload(rl_helper);

## 4.1. Experimentos, análisis y observaciones
En los experimentos a continuación se realizan diversas pruebas, y se van presentando comentarios y observaciones o conclusiones sobre los resultados. Vale mencionar, que en todos los escenarios se aplicó **early stop**, monitoreando el resultado de la función de costo sobre el conjunto de validación se busca parar el proceso de entrenamiento cuando no se detectan mejorías, para evitar tiempos de entrenamiento sin un resultado efectivo. Esto se acompaña con el uso de **model checkpoint**, así se registra el estado del modelo en el mejor resultado registrado, el cual se restaura al final del entrenamiento para evaluar los resultados efectivos.

In [17]:
# Create a list to hold the degree of the polynomial feature used and the performance in train and valid sets,
# elements should be of the format (degree, mae_train, mae_valid)
results = []

### 4.1.1. Regresión lineal
En estas primeras pruebas, se emplea una **regresión lineal** sin ninguna modificación, agregado, técnica o método especial. Es decir, se utiliza un modelo con una **capa densa de una única neurona o unidad**, que será la salida del modelo, con una función de activación **lineal** y una función de costo **MAE**. Por defecto, inicialmente se entrena utilizando como optimizador **SGD**.

#### Observaciones y conclusiones
Durante el entrenamiento del modelo, inicialmente la función de costo es muy elevada y además resulta un proceso muy lento, con lo cual la convergencia a una solución óptima requiere mucho tiempo. Esto se debe particularmente al uso del valor medio del error absoluto (**MAE**) como función de costo, cuya gradiente suele ser menor que otros como puede ser el caso del **MSE**.

In [18]:
# Run model experiment
mae_train, mae_valid, mae_test = rl_helper.run_model(x_train, y_train, x_valid, y_valid, x_test, y_test,
                                                     learning_rate=0.1,
                                                     decay_rate=0.01,
                                                     epochs=500,
                                                     batch_size=64
                                                    )

# Register the results for the degree used
results.append([1, mae_train, mae_valid])

Model logs at tb-logs/rl/20210527-210513
Model checkpoints at checkpoints/rl/20210527-210513
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1)                 10        
Total params: 10
Trainable params: 10
Non-trainable params: 0
_________________________________________________________________
[MAE] Train: 12462.3310546875 Valid: 12616.3984375 Test: 13316.673828125


In [19]:
# Run model experiment
mae_train, mae_valid, mae_test = rl_helper.run_model(x_train, y_train, x_valid, y_valid, x_test, y_test,
                                                     learning_rate=1.0,
                                                     decay_rate=0.01,
                                                     epochs=500,
                                                     batch_size=64
                                                    )

# Register the results for the degree used
results.append([1, mae_train, mae_valid])

Model logs at tb-logs/rl/20210527-211429
Model checkpoints at checkpoints/rl/20210527-211429
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 1)                 10        
Total params: 10
Trainable params: 10
Non-trainable params: 0
_________________________________________________________________
[MAE] Train: 8186.2841796875 Valid: 8316.5234375 Test: 8864.5693359375


### 4.1.2. Learning Rate Scheduling
Se busca solucionar la lenta convergencia que se tiene con una función de costo **MAE** al utilizar valores más grandes de **learning rate**, no obstante, es sabido que esto puede producir divergencia u oscilaciones indeseadas del modelo, entonces se propone probar diferentes esquemas dinámicos en donde el valor empleado puede evolucionar el tiempo, particularmente disminuyendo. Así, se empiezan con valores altos de learning rate que va decayendo con el tiemp, permitiendo alcanzar mínimos de forma más rapida y luego converger sin demasaidas oscilaciones.

#### Observaciones y conclusiones
Se puede notar una diferencia fuerte en la cantidad de tiempo requerido para entrenar el modelo, gracias al uso del **learning rate scheduling**. Además, eso permitió encontrar resultados mucho mejores. Particularmente, no se detectaron diferencias grandes en el resultado respecto del tipo de variación dinámica que se emplea. Se probó utilizar además el optimizador **adam**, y se vió que la curva de aprendizaje presentaba más variación en forma de picos y valles.

Finalmente, resulta de interés observar que la performance del modelo en entrenamiento está por debajo de su performance en validación. Creemos que esto está evidenciando un posible **underfitting** del modelo, ya que su mejor resultado suele esperarse en entrenamiento, dado que aprende a partir de esa información.

In [20]:
# Run model experiment
mae_train, mae_valid, mae_test = rl_helper.run_model(x_train, y_train, x_valid, y_valid, x_test, y_test,
                                                     learning_rate=1000,
                                                     scheduler='time-decay',
                                                     decay_rate=0.01,
                                                     epochs=500,
                                                     batch_size=64
                                                    )

# Register the results for the degree used
results.append([1, mae_train, mae_valid])

Model logs at tb-logs/rl/20210527-211934
Model checkpoints at checkpoints/rl/20210527-211934
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 1)                 10        
Total params: 10
Trainable params: 10
Non-trainable params: 0
_________________________________________________________________
[MAE] Train: 3393.247314453125 Valid: 3543.24658203125 Test: 3378.220947265625


In [21]:
# Run model experiment
mae_train, mae_valid, mae_test = rl_helper.run_model(x_train, y_train, x_valid, y_valid, x_test, y_test,
                                                     learning_rate=1.0,
                                                     scheduler='time-decay',
                                                     decay_rate=0.01,
                                                     epochs=500,
                                                     batch_size=64
                                                    )

Model logs at tb-logs/rl/20210527-212116
Model checkpoints at checkpoints/rl/20210527-212116
Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 1)                 10        
Total params: 10
Trainable params: 10
Non-trainable params: 0
_________________________________________________________________
[MAE] Train: 11011.0810546875 Valid: 11154.4306640625 Test: 11837.4287109375


In [22]:
# Run model experiment
mae_train, mae_valid, mae_test = rl_helper.run_model(x_train, y_train, x_valid, y_valid, x_test, y_test,
                                                     learning_rate=1000,
                                                     scheduler='time-decay',
                                                     decay_rate=0.01,
                                                     optimizer='adam',
                                                     beta_1=0.9,
                                                     beta_2=0.99,
                                                     epochs=500,
                                                     batch_size=64
                                                    )

Model logs at tb-logs/rl/20210527-212620
Model checkpoints at checkpoints/rl/20210527-212620
Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 1)                 10        
Total params: 10
Trainable params: 10
Non-trainable params: 0
_________________________________________________________________
[MAE] Train: 3391.76611328125 Valid: 3551.544677734375 Test: 3356.74853515625


In [ ]:
# Run model experiment
mae_train, mae_valid, mae_test = rl_helper.run_model(x_train, y_train, x_valid, y_valid, x_test, y_test,
                                                     learning_rate=1000,
                                                     scheduler='step-decay',
                                                     drop_rate=0.5,
                                                     epochs_drop=10,
                                                     epochs=500,
                                                     batch_size=32,
                                                    )

Model logs at tb-logs/rl/20210527-212809
Model checkpoints at checkpoints/rl/20210527-212809
Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 1)                 10        
Total params: 10
Trainable params: 10
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Run model experiment
mae_train, mae_valid, mae_test = rl_helper.run_model(x_train, y_train, x_valid, y_valid, x_test, y_test,
                                                     learning_rate=1000,
                                                     scheduler='step-decay',
                                                     drop_rate=0.5,
                                                     optimizer='adam',
                                                     beta_1=0.9,
                                                     beta_2=0.99,
                                                     epochs_drop=10,
                                                     epochs=500,
                                                     batch_size=32,
                                                    )

In [ ]:
# Run model experiment
mae_train, mae_valid, mae_test = rl_helper.run_model(x_train, y_train, x_valid, y_valid, x_test, y_test,
                                                     learning_rate=1000,
                                                     scheduler='exponential-decay',
                                                     decay_rate=0.07,
                                                     epochs=500,
                                                     batch_size=32,
                                                    )

In [ ]:
# Run model experiment
mae_train, mae_valid, mae_test = rl_helper.run_model(x_train, y_train, x_valid, y_valid, x_test, y_test,
                                                     learning_rate=1000,
                                                     scheduler='exponential-decay',
                                                     decay_rate=0.01,
                                                     optimizer='adam',
                                                     beta_1=0.9,
                                                     beta_2=0.99,
                                                     min_delta=1,
                                                     epochs=500,
                                                     batch_size=32,
                                                    )

### 4.1.3. Polynomial Features
Se sospecha que la diferencia entre la performance de entrenamiento y validación es una manifestación de un **underfitting** del modelo, es probable que la complejidad del modelo no sea lo suficientemente alta para poder representar correctamente la solución al problema, una posible solución es utilizar **polynomial features** que permite crear un espacio de soluciones más grande. A continuación, se prueba con diferentes órdenes y se observan resultados.

#### Observaciones y conclusiones
1. Se puede observar que para órdenes bajos, pero mayores a uno, sigue habiendo una diferencia notable entre entrenamiento y validación.
2. En general, los resultados son mucho mejores que con el escenario anterior de primer orden.
3. A medida que aumentamos el orden del feature polinomial, se observan pocas variaciones en la métrica de entrenamiento, pero se ve un incremento en la métrica de validación. Esto quiere decir que su resultado está empeorando en validación y acercándose al entrenamiento, disminuyendo así el **underfitting**. No obstante, puede valorarse como algo positivo, ya que métricas similares en entrenamiento y validación es un buen indicio de la capacidad de **generalización** del modelo. Es decir, no sólo sabriamos que el resultado es bueno sino que además podemos esperar una buena generalización para datos sobre los cuales no se entrenó.
4. A medida que elevamos más el orden del feature polinomial, se puede observar que empiezan a suceder situaciones de **overfitting**, el modelo nuevamente pierde capacidad de generalización, pero esta vez porque su performance en entrenamiento da mucho mejor que en el resto de los casos. Esto evidencia que aprendió demasiado sobre entrenamiento. En este punto, deberíamos conseguir más datos para permitirle generalizar mejor, o limitar el gran espacio de soluciones que posee frente a la gran complejidad por ser de alto orden.
5. Observar que para ordenes muy grandes fue necesario reducir el batch size para poder encontrar una buena solución.
6. Observar que el learning rate empleado ahora es menor.

In [ ]:
# Run model experiment
mae_train, mae_valid, mae_test = rl_helper.run_model(x_train, y_train, x_valid, y_valid, x_test, y_test,
                                                     learning_rate=1000,
                                                     degree=2,
                                                     scheduler='exponential-decay',
                                                     decay_rate=0.09,
                                                     optimizer='adam',
                                                     beta_1=0.9,
                                                     beta_2=0.99,
                                                     patience=50,
                                                     min_delta=1,
                                                     epochs=1000,
                                                     batch_size=64
                                                    )

# Register the results for the degree used
results.append([2, mae_train, mae_valid])

In [ ]:
# Run model experiment
mae_train, mae_valid, mae_test = rl_helper.run_model(x_train, y_train, x_valid, y_valid, x_test, y_test,
                                                     learning_rate=1000,
                                                     degree=3,
                                                     scheduler='exponential-decay',
                                                     decay_rate=0.1,
                                                     optimizer='adam',
                                                     beta_1=0.9,
                                                     beta_2=0.99,
                                                     patience=50,
                                                     min_delta=1,
                                                     epochs=1000,
                                                     batch_size=64
                                                    )

# Register the results for the degree used
results.append([3, mae_train, mae_valid])

In [ ]:
# Run model experiment
mae_train, mae_valid, mae_test = rl_helper.run_model(x_train, y_train, x_valid, y_valid, x_test, y_test,
                                                     learning_rate=1000,
                                                     degree=4,
                                                     scheduler='exponential-decay',
                                                     decay_rate=0.1,
                                                     optimizer='adam',
                                                     beta_1=0.9,
                                                     beta_2=0.99,
                                                     patience=50,
                                                     min_delta=1,
                                                     epochs=1000,
                                                     batch_size=64
                                                    )

# Register the results for the degree used
results.append([4, mae_train, mae_valid])

In [ ]:
# Run model experiment
mae_train, mae_valid, mae_test = rl_helper.run_model(x_train, y_train, x_valid, y_valid, x_test, y_test,
                                                     learning_rate=2.0,
                                                     degree=5,
                                                     scheduler='exponential-decay',
                                                     decay_rate=0.001,
                                                     optimizer='adam',
                                                     beta_1=0.9,
                                                     beta_2=0.99,
                                                     patience=50,
                                                     min_delta=1,
                                                     epochs=1000,
                                                     batch_size=64
                                                    )

# Register the results for the degree used
results.append([5, mae_train, mae_valid])

In [ ]:
# Run model experiment
mae_train, mae_valid, mae_test = rl_helper.run_model(x_train, y_train, x_valid, y_valid, x_test, y_test,
                                                     learning_rate=5.0,
                                                     degree=6,
                                                     scheduler='exponential-decay',
                                                     decay_rate=0.01,
                                                     optimizer='adam',
                                                     beta_1=0.9,
                                                     beta_2=0.99,
                                                     patience=50,
                                                     min_delta=1,
                                                     epochs=1000,
                                                     batch_size=32
                                                    )

# Register the results for the degree used
results.append([6, mae_train, mae_valid])

In [ ]:
# Run model experiment
mae_train, mae_valid, mae_test = rl_helper.run_model(x_train, y_train, x_valid, y_valid, x_test, y_test,
                                                     learning_rate=5.0,
                                                     degree=7,
                                                     scheduler='exponential-decay',
                                                     decay_rate=0.01,
                                                     optimizer='adam',
                                                     beta_1=0.9,
                                                     beta_2=0.99,
                                                     patience=50,
                                                     min_delta=10,
                                                     epochs=1000,
                                                     batch_size=32
                                                    )

# Register the results for the degree used
results.append([7, mae_train, mae_valid])

In [ ]:
# Run model experiment
mae_train, mae_valid, mae_test = rl_helper.run_model(x_train, y_train, x_valid, y_valid, x_test, y_test,
                                                     learning_rate=5.0,
                                                     degree=8,
                                                     scheduler='exponential-decay',
                                                     decay_rate=0.01,
                                                     optimizer='adam',
                                                     beta_1=0.9,
                                                     beta_2=0.99,
                                                     patience=50,
                                                     min_delta=10,
                                                     epochs=1000,
                                                     batch_size=32
                                                    )

# Register the results for the degree used
results.append([8, mae_train, mae_valid])

In [ ]:
# Run model experiment
mae_train, mae_valid, mae_test = rl_helper.run_model(x_train, y_train, x_valid, y_valid, x_test, y_test,
                                                     learning_rate=5.0,
                                                     degree=9,
                                                     scheduler='exponential-decay',
                                                     decay_rate=0.01,
                                                     optimizer='adam',
                                                     beta_1=0.9,
                                                     beta_2=0.99,
                                                     patience=50,
                                                     min_delta=10,
                                                     epochs=1000,
                                                     batch_size=32
                                                    )

# Register the results for the degree used
results.append([9, mae_train, mae_valid])

In [ ]:
# Run model experiment
mae_train, mae_valid, mae_test = rl_helper.run_model(x_train, y_train, x_valid, y_valid, x_test, y_test,
                                                     learning_rate=5.0,
                                                     degree=10,
                                                     scheduler='exponential-decay',
                                                     decay_rate=0.001,
                                                     optimizer='adam',
                                                     beta_1=0.9,
                                                     beta_2=0.99,
                                                     patience=50,
                                                     min_delta=10,
                                                     epochs=1000,
                                                     batch_size=32
                                                    )

# Register the results for the degree used
results.append([10, mae_train, mae_valid])

#### Análisis de la métrica en función del orden de feature polinomial
Utilizando el registro de los resultados para cada orden de feature polinomial,se grafica la evolución para observar el comportamiento y sacar conclusiones a partir de ello.

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# Decoding results into its components
results_orders = [order for order, mae_train, mae_valid in results]
results_mae_train = [mae_train for order, mae_train, mae_valid in results]
results_mae_valid = [mae_valid for order, mae_train, mae_valid in results]

# Plot each of them with a color and a label
plt.plot(results_orders, results_mae_train, color='blue', label='Train')
plt.plot(results_orders, results_mae_valid, color='orange', label='Valid')
plt.xlabel('Orden')
plt.ylabel('MAE')
plt.legend()
plt.grid()
plt.show()

### 4.1.3. Regularización
A partir de los resultados anteriores, se observa que a medida que aumentamos el orden del feature polinomial el modelo es capaz de resover mejor el problema, no sólo mejorando la métrica, sino generalizando mejor. No obstante, cuando se eleva demasiado el orden del polinomio se empieza a enfrentar un problema de overfitting. Esto es, que el modelo presenta un espacio de soluciones de muy alta dimensionalidad que le permite adaptarse flexiblemente al comportamiento del conjunto de entrenamiento, entonces aunque esto no implique que la métrica de entrenamiento mejore, sí implica que la solución obtenida está muy arraigada a la naturaleza del conjunto de entrenamiento.
El objetivo ahora es emplear **regularización** para poder restringir el espacio de soluciones, utilizando los métodos **L1** y **L2**, para ver si se puede reducir el overfitting.

#### Observaciones y conclusiones
Efectivamente se nota una mejoría en la reducción del overfitting, particularmente para el caso de regularización L2.

In [ ]:
# Run model experiment
mae_train, mae_valid, mae_test = rl_helper.run_model(x_train, y_train, x_valid, y_valid, x_test, y_test,
                                                     learning_rate=5.0,
                                                     degree=9,
                                                     regularizer='l1',
                                                     regularizer_lambda=1e-4,
                                                     scheduler='exponential-decay',
                                                     decay_rate=0.01,
                                                     optimizer='adam',
                                                     beta_1=0.9,
                                                     beta_2=0.99,
                                                     patience=50,
                                                     min_delta=10,
                                                     epochs=1000,
                                                     batch_size=32
                                                    )

In [ ]:
# Run model experiment
mae_train, mae_valid, mae_test = rl_helper.run_model(x_train, y_train, x_valid, y_valid, x_test, y_test,
                                                     learning_rate=5.0,
                                                     degree=9,
                                                     regularizer='l2',
                                                     regularizer_lambda=1e-4,
                                                     scheduler='exponential-decay',
                                                     decay_rate=0.01,
                                                     optimizer='adam',
                                                     beta_1=0.9,
                                                     beta_2=0.99,
                                                     patience=50,
                                                     min_delta=10,
                                                     epochs=1000,
                                                     batch_size=32
                                                    )

### 4.2. Conclusiones
La conclusión final luego de haber realizado estos experimentos, es que si se desea resolver el problema, se debería hacer una selección del modelo variando como hiperparámetros el **orden del feature polinomial**, el **learning rate**, **el batch size** y el **coeficiente de regularización L2**. Creemos que, partiendo esos hiperparámetros, y utilizando algún método de búsqueda como puede ser la búsqueda bayesiana, se puede llegar a un modelo "bueno" capaz de generalizar. Aunque esta conclusión parezca evidente, debe notarse que no fue hasta realizar todos estos experimentos y analizar sus resultados, que pudimos convencernos de que ese era el camino.